In [ ]:
import pandas as pd
import numpy as np
# from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error,accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from joblib import parallel_backend


In [ ]:
# Define the function to map months to seasons in New York
def map_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'Autumn'

In [ ]:
# Function to expand monthly totals into daily values
def expand_monthly_to_daily(df, date_col, value_cols, name_col, season_col):
    daily_data = []
    for _, row in df.iterrows():
        month_days = pd.date_range(
            start=row[date_col].replace(day=1),
            end=row[date_col] + pd.offsets.MonthEnd(0),
            freq='D'
        )
        daily_values = {col: row[col] / len(month_days) for col in value_cols}
        for day in month_days:
            daily_data.append({
                date_col: day,
                name_col: row[name_col],
                season_col: row[season_col],  # Include the season
                **daily_values
            })
    return pd.DataFrame(daily_data)

In [ ]:
# Load the dataset
df = pd.read_csv("2021_2022_NewYork_Sales_Cancel.csv", encoding='unicode_escape')
df['ds'] = pd.to_datetime(df['years'].astype(str) + '-' + df['months'].astype(str))
df['season'] = df['months'].apply(map_season)
df['season'] = df['season'].astype('category').cat.codes

In [ ]:
df[df['ShortName'] == 'Hot Mix 12oz.']

,item_id,item_code,ShortName,years,months,saleamt,saleqty,cancelamt,cancelqty,ds,season
0,1,11,Hot Mix 12oz.,2021,1,0.00000,0.0,55362.83,19940.0,2021-01-01,3
4330,1,11,Hot Mix 12oz.,2021,2,25854.96070,7540.0,32268.97,11680.0,2021-02-01,3
8660,1,11,Hot Mix 12oz.,2021,3,32305.73290,8300.0,8141.02,2940.0,2021-03-01,1
12990,1,11,Hot Mix 12oz.,2021,4,55.79000,20.0,38488.46,13900.0,2021-04-01,1
17320,1,11,Hot Mix 12oz.,2021,5,598.90000,140.0,49498.45,17060.0,2021-05-01,1
21650,1,11,Hot Mix 12oz.,2021,6,32452.72050,8760.0,27585.38,9240.0,2021-06-01,2
25980,1,11,Hot Mix 12oz.,2021,7,27924.01160,6520.0,9422.94,3140.0,2021-07-01,2
30310,1,11,Hot Mix 12oz.,2021,8,25781.20030,6540.0,19110.51,6400.0,2021-08-01,2
34640,1,11,Hot Mix 12oz.,2021,9,3268.21620,740.0,25053.24,8420.0,2021-09-01,0
38970,1,11,Hot Mix 12oz.,2021,10,29848.69650,7940.0,14194.81,4760.0,2021-10-01,0


In [ ]:
for i in df['ShortName'].unique()[:210]:
  df = df[df['ShortName'] == i]  ## change here
  # Expand the monthly totals into daily values
  daily_df = expand_monthly_to_daily(df, 'ds', ['saleqty', 'cancelqty'], 'ShortName', 'season')
  def add_time_features(df, date_col):
      df['year'] = df[date_col].dt.year
      df['month'] = df[date_col].dt.month
      df['day'] = df[date_col].dt.day
      # df['day_of_week'] = df[date_col].dt.dayofweek
      return df
  # Add time-related features to the daily data
  daily_df = add_time_features(daily_df, 'ds')

  # Encode 'ShortName' using label encoding
  label_encoder = LabelEncoder()
  daily_df['ShortName_encoded'] = label_encoder.fit_transform(daily_df['ShortName'])
  daily_df.head()
  # Drop the original 'ShortName' and 'ds' columns
  X = daily_df.drop(['ShortName', 'ds', 'saleqty', 'cancelqty'], axis=1)
  y_saleqty = daily_df['saleqty']
  y_cancelqty = daily_df['cancelqty']
  # print(X)
  # print(y_saleqty)
  # Time series cross-validation
  tscv = TimeSeriesSplit(n_splits=100)
  # Initialize models
  saleqty_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
  cancelqty_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
  # Train separate models for 'saleqty' and 'cancelqty'
  for target, model in zip(['saleqty', 'cancelqty'], [saleqty_model, cancelqty_model]):
      y = daily_df[target]

      for train_index, test_index in tscv.split(X):
          X_train, X_test = X.iloc[train_index], X.iloc[test_index]
          y_train, y_test = y.iloc[train_index], y.iloc[test_index]

          # Train the model
          model.fit(X_train, y_train)

          # Predict on test set
          y_pred = model.predict(X_test)

          # Calculate RMSE
          rmse = np.sqrt(mean_squared_error(y_test, y_pred))
          # print(f'RMSE for {target}: {rmse}')

  print("Training done")
  # Prepare the DataFrame for predictions in January 2024
  jan_2024_days = pd.date_range(start='2023-03-01', end='2023-03-31', freq='D') ## change here
  num_days = len(jan_2024_days)
  num_short_names = len(daily_df['ShortName_encoded'].unique())
  # Create repeated day entries for each 'ShortName'
  repeated_days = np.repeat(jan_2024_days, num_short_names)

  # Repeat each 'ShortName_encoded' for each day in January 2024
  repeated_short_names = np.tile(daily_df['ShortName_encoded'].unique(), num_days)
  # Repeat the season value for January for each day and each 'ShortName'
  season_value = map_season(3)  # January is winter
  repeated_seasons = np.tile(season_value, num_days * num_short_names)

  future_data = {
      'ds': repeated_days,
      'ShortName_encoded': repeated_short_names
      # 'season': repeated_seasons
  }
  future_df = pd.DataFrame(future_data)
  # future_df['months'] = 1
  # future_df['season'] = future_df['months'].apply(map_season)
  # future_df['season'] = future_df['season'].astype('category').cat.codes
  future_df['season'] = df[df['months'] == 3]['season'].values[0] ##change here
  # print(future_df)
  # Convert 'ds' into numerical features
  future_df = add_time_features(future_df, 'ds')
  # future_df['season'] = future_df['season'].astype('category').cat.codes

  # Drop the original 'ds' datetime column
  future_df.drop(['ds'], axis=1, inplace=True)

  feature_columns = ['season', 'year', 'month', 'day', 'ShortName_encoded']  # Add any other features used during training
  future_df = future_df[feature_columns]
  # print(future_df)
  future_df_for_saleqty = future_df[feature_columns]
  future_df['predicted_saleqty'] = saleqty_model.predict(future_df_for_saleqty)
  # Predict cancelqty
  future_df_for_cancelqty = future_df[feature_columns]  # Do not include 'predicted_saleqty'
  future_df['predicted_cancelqty'] = cancelqty_model.predict(future_df_for_cancelqty)

  # Aggregate the daily predictions to get the total predicted sales for each `ShortName` in January 2024
  future_df['ShortName'] = label_encoder.inverse_transform(future_df['ShortName_encoded'])
  aggregated_predictions = future_df.groupby('ShortName').agg({
      'predicted_saleqty': 'sum',
      'predicted_cancelqty': 'sum'
  }).reset_index()

  # aggregated_predictions.to_csv('aggregated_predictions_January_2024.csv', index=False)

  # print("Aggregated predictions for January 2024 saved to CSV.")
  len(set(aggregated_predictions['ShortName'].to_list()))
  aggregated_predictions['predicted_saleqty'] = aggregated_predictions['predicted_saleqty'] - aggregated_predictions['predicted_cancelqty']
  aggregated_predictions.drop('predicted_cancelqty', axis=1, inplace=True)
  aggregated_predictions
  new_df = pd.DataFrame(columns=list(aggregated_predictions.columns).extend(['2021_saleqty', '2022_saleqty']))
  for i,temp_df in aggregated_predictions.iterrows():
    temp_df['2021_saleqty'] = df[df['ShortName'].isin([temp_df['ShortName']]) & (df['years'] == 2021) & (df['months']==3)]['saleqty'].values[0] ## change here month
    temp_df['2022_saleqty'] = df[df['ShortName'].isin([temp_df['ShortName']]) & (df['years'] == 2022) & (df['months']==3)]['saleqty'].values[0] ## change here month
    temp_df['2021_cancelqty'] = df[df['ShortName'].isin([temp_df['ShortName']]) & (df['years'] == 2021) & (df['months']==3)]['cancelqty'].values[0] ## change here month
    temp_df['2022_cancelqty'] = df[df['ShortName'].isin([temp_df['ShortName']]) & (df['years'] == 2022) & (df['months']==3)]['cancelqty'].values[0] ## change here month
    new_df = new_df.append(temp_df)
  new_df
  # Load the existing CSV file into a DataFrame
  csv_file_path = 'predictions_March_2023_RFS.csv'
  existing_dataframe = pd.read_csv(csv_file_path)

  # Convert the new data to a DataFrame
  new_dataframe = pd.DataFrame(new_df)

  # Append the new rows to the existing DataFrame
  updated_dataframe = existing_dataframe.append(new_dataframe, ignore_index=True)

  # Save the updated DataFrame back to the CSV file
  updated_dataframe.to_csv(csv_file_path, index=False)

  df = pd.read_csv("2021_2022_NewYork_Sales_Cancel.csv", encoding='unicode_escape')
  df['ds'] = pd.to_datetime(df['years'].astype(str) + '-' + df['months'].astype(str))
  df['season'] = df['months'].apply(map_season)
  df['season'] = df['season'].astype('category').cat.codes


Training done


<ipython-input-14-e1afc345188f>:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(temp_df)
<ipython-input-14-e1afc345188f>:118: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  updated_dataframe = existing_dataframe.append(new_dataframe, ignore_index=True)


Training done


<ipython-input-14-e1afc345188f>:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(temp_df)
<ipython-input-14-e1afc345188f>:118: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  updated_dataframe = existing_dataframe.append(new_dataframe, ignore_index=True)


Training done


...

<ipython-input-14-e1afc345188f>:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(temp_df)
<ipython-input-14-e1afc345188f>:118: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  updated_dataframe = existing_dataframe.append(new_dataframe, ignore_index=True)


Training done


<ipython-input-14-e1afc345188f>:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(temp_df)
<ipython-input-14-e1afc345188f>:118: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  updated_dataframe = existing_dataframe.append(new_dataframe, ignore_index=True)


Training done


<ipython-input-14-e1afc345188f>:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(temp_df)
<ipython-input-14-e1afc345188f>:118: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  updated_dataframe = existing_dataframe.append(new_dataframe, ignore_index=True)
